In [1]:
import os

from cnn_finetune import make_model
import pandas as pd
from skimage import io, transform
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms, utils

/Users/kyohei.watarai/.pyenv/versions/3.7.0/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [2]:
model = make_model('resnet18', num_classes=2, pretrained=True, input_size=(128, 128))
device = torch.device('cpu')
model = model.to(device)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=model.original_model_info.mean,
        std=model.original_model_info.std),
])

In [4]:
class MyDataSet(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data_df = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self, idx):
        image_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
        image = io.imread(image_name)
        label = self.data_df.iloc[idx, 1]
        return self.transform(image), int(label)

In [5]:
train_set = MyDataSet('../images/train_data.csv', '../images/resize/data/', transform)
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True)

In [6]:
test_set = MyDataSet('../images/test_data.csv', '../images/resize/data/', transform)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True)

https://github.com/creafz/pytorch-cnn-finetune/blob/master/examples/cifar10.py

In [7]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [22]:
def train(epoch):
    total_loss = 0
    total_size = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        
        output_logit = output.detach().numpy()
        print(output_logit.shape)
        
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 20 == 0 or (batch_idx+1)==len(train_loader):
            progress = 100. * (batch_idx+1) / len(train_loader)
            average_loss = total_loss / total_size
            print(f'Train Epoch: {epoch} [{(batch_idx+1) * len(data)}/{len(train_loader.dataset)} ({progress:.0f}%)]\tAverage loss: {average_loss:.6f}')

In [23]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

In [24]:
for epoch in range(1, 6):
    train(epoch)
    test()

(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
Train Epoch: 1 [20/173 (12%)]	Average loss: 3.578607
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)


KeyboardInterrupt: 